
# 03 — Feature Engineering  
This notebook extracts statistical, time‑domain, and frequency‑domain features  
from aligned and filtered WESAD physiological signals.

You will replace the placeholder data with the real aligned arrays.


In [14]:
# ============================================================
# STEP-3 — Feature Engineering (ALL SUBJECTS)
# Physiological + Behavioral Features
# ============================================================

import numpy as np
import os
from scipy.stats import skew, kurtosis

# -----------------------------
# CONFIG
# -----------------------------
SUBJECT_IDS = [
    2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 13, 14, 15, 17
]

WINDOW_SIZE = 60
INPUT_TEMPLATE  = "aligned_windows_S{}_w60.npz"
OUTPUT_TEMPLATE = "features_S{}_w60.npz"

# -----------------------------
# Feature extractors
# -----------------------------
def extract_stats(x):
    """Basic statistical features"""
    return np.stack([
        x.mean(axis=1),
        x.std(axis=1),
        np.min(x, axis=1),
        np.max(x, axis=1),
        skew(x, axis=1),
        kurtosis(x, axis=1)
    ], axis=1)

def extract_behavioral(acc):
    """
    Behavioral features from ACC
    acc shape: (N, T)
    """
    energy = np.mean(acc**2, axis=1)
    motion = np.std(acc, axis=1)
    peak   = np.max(np.abs(acc), axis=1)
    zcr    = np.mean(np.diff(np.sign(acc), axis=1) != 0, axis=1)

    return np.stack([energy, motion, peak, zcr], axis=1)

# -----------------------------
# MAIN LOOP
# -----------------------------
processed = 0

for sid in SUBJECT_IDS:
    in_file  = INPUT_TEMPLATE.format(sid)
    out_file = OUTPUT_TEMPLATE.format(sid)

    if not os.path.exists(in_file):
        print(f"⚠️ Missing Step-2 file for S{sid}, skipping")
        continue

    data = np.load(in_file)

    EDA  = data["EDA_windows"]
    BVP  = data["BVP_windows"]
    ACC  = data["ACC_windows"]
    TEMP = data["TEMP_windows"]
    labels = data["label_windows"]  # {1,2,3}

    # -----------------------------
    # Physiological features
    # -----------------------------
    EDA_f  = extract_stats(EDA)
    BVP_f  = extract_stats(BVP)
    ACC_f  = extract_stats(ACC)
    TEMP_f = extract_stats(TEMP)

    phys_features = np.concatenate(
        [EDA_f, BVP_f, ACC_f, TEMP_f], axis=1
    )

    # -----------------------------
    # Behavioral features
    # -----------------------------
    beh_features = extract_behavioral(ACC)

    # -----------------------------
    # Save
    # -----------------------------
    np.savez(
        out_file,
        EDA_windows=EDA,
        BVP_windows=BVP,
        ACC_windows=ACC,
        TEMP_windows=TEMP,
        phys_features=phys_features,
        behavior_features=beh_features,
        labels=labels
    )

    processed += 1
    print(f"✓ Subject S{sid} processed → {out_file}")

# -----------------------------
# DONE
# -----------------------------
print(f"\n🎉 Step-3 complete | Subjects processed: {processed}")


✓ Subject S2 processed → features_S2_w60.npz
✓ Subject S3 processed → features_S3_w60.npz
✓ Subject S4 processed → features_S4_w60.npz
✓ Subject S5 processed → features_S5_w60.npz
✓ Subject S6 processed → features_S6_w60.npz
✓ Subject S7 processed → features_S7_w60.npz
✓ Subject S8 processed → features_S8_w60.npz
✓ Subject S9 processed → features_S9_w60.npz
✓ Subject S10 processed → features_S10_w60.npz
✓ Subject S11 processed → features_S11_w60.npz
✓ Subject S13 processed → features_S13_w60.npz
✓ Subject S14 processed → features_S14_w60.npz
✓ Subject S15 processed → features_S15_w60.npz
✓ Subject S17 processed → features_S17_w60.npz

🎉 Step-3 complete | Subjects processed: 14


C:\Users\Barathirajan M\AppData\Local\Temp\ipykernel_7276\3964657231.py:32: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew(x, axis=1),
C:\Users\Barathirajan M\AppData\Local\Temp\ipykernel_7276\3964657231.py:33: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis(x, axis=1)
